<a href="https://colab.research.google.com/github/neel0086/deep_learning/blob/main/CustomerChurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import io
%matplotlib inline

In [18]:
df = pd.read_csv(io.BytesIO(uploaded['customer_churn.csv']))
df.drop('customerID',axis='columns',inplace=True)
df = pd.DataFrame(df)

df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [24]:
# REMOVING NOISE FROM THE DATA #
df1=df[df.TotalCharges!=' ']
df1.shape

(7032, 20)

In [26]:
# Change to numeric data for Total Charges
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

<ipython-input-26-e5a14620678c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.TotalCharges = pd.to_numeric(df1.TotalCharges)


In [1]:
tenure_churn_no = df1[df1.Churn == "No"].tenure
tenure_churn_yes = df1[df1.Churn == "Yes"].tenure

plt.hist([tenure_churn_yes, tenure_churn_no],color=['green','red'],label=['Churn=Yes','Churn='No])

SyntaxError: ignored